# Part I. ETL Pipeline for Pre-Processing the Files

## PRE-PROCESS THE DATA FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

from cassandra.cluster import Cluster
from IPython.display import HTML

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking the current working directory
print(os.getcwd())

# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/hynso/Documents/Learning/Udacity/DataEngineering/1-DataModeling/Lesson5


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# Initiate an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# For every filepath in the file path list 
for f in file_path_list:
    # Read csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # Create a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # Extract each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# Create a smaller event data csv file called event_datafile_new.csv 
# that will be used to insert data into the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                     'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
# check the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra coding portion of your project. 

## The event_datafile_new.csv contains the following columns: 

- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the `event_datafile_new.csv` after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# Create a cluster object and connect to it with a "session"
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
# Create a keyspace called `udacity`
try:
    session.execute(
        """
        CREATE KEYSPACE IF NOT EXISTS udacity
        WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
        """
    )
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set the working keyspace to the freshly made `udacity` keyspace
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### Query 1: Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

Query Description: Here the `session_id` and `item_in_session` fields will be used in the WHERE clause, so these two fields should (1) make up the composite primary key and (2) make up the composite partition key. As a composite primary key, these fields will allow each record in the table to be uniquely identified. They weren't originally part of a composite partition key, but I received feedback that they should be. However, I don't fully understand the rationale.

In [8]:
# Create the table for this query

query = \
    """
    CREATE TABLE IF NOT EXISTS artist_song_table (
      session_id      INT, -- partition key column 0
      item_in_session INT, -- partition key column 1
      artist_name     TEXT,
      song_title      TEXT,
      song_length     DOUBLE,
      PRIMARY KEY ((session_id, item_in_session))
    )
    """

try:
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
# Insert the appropriate data based on the data model

file = 'event_datafile_new.csv'

query = \
    """
    INSERT INTO artist_song_table (
      session_id,
      item_in_session,
      artist_name,
      song_title,
      song_length
    )
    VALUES (%s, %s, %s, %s, %s)
    """

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:     
        session.execute(
          query, 
          (int(line[8]), int(line[3]), line[0], line[9], float(line[5]))
        )

In [10]:
# Do a SELECT to verify the data model and 
# that correct data has been inserted into the table

query = \
    """
    SELECT
      artist_name, 
      song_title, 
      song_length 
    FROM artist_song_table 
    WHERE
      session_id=338 AND
      item_in_session=4
    """

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

HTML(pd.DataFrame(rows).to_html(index=False))

artist_name,song_title,song_length
Faithless,Music Matters (Mark Knight Dub),495.3073


### Query 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

Query Description: Here the `user_id` and `session_id` fields will be used in the WHERE clause, so these two fields should (1) make up _**part**_ of the composite primary key and (2) make up the composite partition key. In addition to `user_id` and `session_id`, the `item_in_session` field will be included as the final part of the composite primary key. As a composite primary key, these three fields will allow each record in the table to be uniquely identified. As a composite partition key, both `user_id` and `session_id` will allow partitions to be appropriately sized for performant queries.

In [11]:
# Create the table for this query

query = \
    """
    CREATE TABLE IF NOT EXISTS artist_song_user_table (
      user_id         INT,  -- partition key column 0
      session_id      INT,  -- partition key column 1
      item_in_session INT,  -- clustering column 0
      artist_name     TEXT,
      song_title      TEXT,
      first_name      TEXT,
      last_name       TEXT,
      PRIMARY KEY ((user_id, session_id), item_in_session)
    )
    """

try:
    session.execute(query)
except Exception as e:
    print(e)

In [12]:
# Insert the appropriate data based on the data model

file = 'event_datafile_new.csv'

query = \
    """
    INSERT INTO artist_song_user_table (
      user_id,
      session_id,
      item_in_session,
      artist_name,
      song_title,
      first_name,
      last_name
    )
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    """

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        session.execute(
          query, 
          (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4])
        )

In [13]:
# Do a SELECT to verify the data model and 
# that correct data has been inserted into the table

query = \
    """
    SELECT
      artist_name, 
      song_title, 
      first_name,
      last_name,
      item_in_session
    FROM artist_song_user_table 
    WHERE 
      user_id=10 AND
      session_id=182
    """

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

HTML(pd.DataFrame(rows).to_html(index=False))

artist_name,song_title,first_name,last_name,item_in_session
Down To The Bone,Keep On Keepin' On,Sylvie,Cruz,0
Three Drives,Greece 2000,Sylvie,Cruz,1
Sebastien Tellier,Kilometer,Sylvie,Cruz,2
Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio Edit),Sylvie,Cruz,3


### Query 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

Query Description: Here the `song_title` field will be used in the WHERE clause, so this field should (1) make up _**part**_ of the composite primary key and (2) be the partition key. In addition to `song_title`, the `user_id` field will be included as the final part of the composite primary key. As a composite primary key, these two fields will allow each record in the table to be uniquely identified. A reviewer recommended that both fields make up the composite partition key, but implementing this resulted in the following error message:

```
Error from server: code=2200 [Invalid query] 
message="Partition key parts: user_id must be restricted as other parts are"
```

Therefore, only the `song_title` field has been retained as the sole partition key column.

In [14]:
# Create the table for this query

query = \
    """
    CREATE TABLE IF NOT EXISTS user_song_table (
      song_title      TEXT, -- partition key column 0
      user_id         INT,  -- clustering column 0
      first_name      TEXT,
      last_name       TEXT,
      PRIMARY KEY (song_title, user_id)
    )
    """

try:
    session.execute(query)
except Exception as e:
    print(e)

In [15]:
# Insert the appropriate data based on the data model

file = 'event_datafile_new.csv'

query = \
    """
    INSERT INTO user_song_table (
      song_title,
      user_id,
      first_name,
      last_name
    )
    VALUES (%s, %s, %s, %s)
    """

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        session.execute(
          query, 
          (line[9], int(line[10]), line[1], line[4])
        )

In [16]:
# Do a SELECT to verify the data model and 
# that correct data has been inserted into the table

query = \
    """
    SELECT
      first_name,
      last_name
    FROM user_song_table 
    WHERE song_title='All Hands Against His Own'
    """

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

HTML(pd.DataFrame(rows).to_html(index=False))

first_name,last_name
Jacqueline,Lynch
Tegan,Levine
Sara,Johnson


### Drop the tables before closing out the sessions

In [17]:
# Drop the first table
query = "DROP TABLE IF EXISTS artist_song_table"

try:
    res = session.execute(query)
except Execption as e:
    print(e)
    
# Drop the second table
query = "DROP TABLE IF EXISTS artist_song_user_table"

try:
    res = session.execute(query)
except Execption as e:
    print(e)

# Drop the third table
query = "DROP TABLE IF EXISTS user_song_table"

try:
    res = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection

In [18]:
session.shutdown()
cluster.shutdown()